In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# import dataset roomstatinfo.csv
room_stat = pd.read_csv('roomstatinfo.csv')
room_stat.head()

In [ ]:
# change datetime column to timestamp, groupby week,count meeting times
room_stat["startTime"] = pd.to_datetime(room_stat['startTime'])
group_room_stat = room_stat.set_index('startTime').groupby([ "CompanyName"]).resample("W").size().to_frame(name = 'count').reset_index()
#group_room_stat

In [ ]:
chair = pd.read_csv('roomlogchair.csv')
#chair.head()

In [ ]:
# change datetime column to timestamp, groupby week,count meeting times
chair["EnterTime"] = pd.to_datetime(chair["EnterTime"])
group_chair = chair.set_index('EnterTime').groupby(["CompanyName"]).resample("W").size().to_frame(name = 'count').reset_index()
#group_chair

In [ ]:
user = pd.read_csv('userlog.csv')
#user.head()

In [ ]:
# change datetime column to timestamp, groupby week,count meeting times
user["LogTime"] = pd.to_datetime(user['LogTime'])
group_user = user.set_index('LogTime').groupby(["CompanyName"]).resample("W").size().to_frame(name = 'count').reset_index()
#group_user

In [ ]:
# merge room_stat df with user df
group_stat_user_merge = pd.merge(group_room_stat,group_user,how = 'left',left_on = ['CompanyName','startTime'],right_on = ['CompanyName','LogTime'])
#group_stat_user_merge

In [ ]:
# merge again with chair df
group_merge_all = pd.merge(group_stat_user_merge,group_chair,how = 'left',left_on = ['CompanyName','startTime'],right_on = ['CompanyName','EnterTime'])
#group_merge_all

In [ ]:
group_merge_all.rename(index=str, columns={"count_x": "meeting_count", "count_y": "key_user_count","count": "chair_count"},inplace=True)
#group_merge_all

In [ ]:
group_merge_all.rename(index=str, columns={"meeting_count": "会议次数", "key_user_count": "操作员登陆次数","chair_count": "主席登陆次数"},inplace=True)
#group_merge_all

In [ ]:
# change datetime column to string
group_merge_all['startTime'] = group_merge_all['startTime'].apply(str)
group_merge_all['LogTime'] = group_merge_all['LogTime'].apply(str)
group_merge_all['EnterTime'] = group_merge_all['EnterTime'].apply(str)
type(group_merge_all['EnterTime'][10])

In [ ]:
# replace 0 time
group_merge_all['startTime'] = group_merge_all['startTime'].str.replace(' 00:00:00','')
group_merge_all['LogTime'] = group_merge_all['LogTime'].str.replace(' 00:00:00','')
group_merge_all['EnterTime'] = group_merge_all['EnterTime'].str.replace(' 00:00:00','')
group_merge_all['EnterTime'] = group_merge_all['EnterTime'].str.replace('NaT','0')
group_merge_all['LogTime'] = group_merge_all['LogTime'].str.replace('NaT','0')
#group_merge_all

In [ ]:
# impute NaT NaN
group_merge_all['主席登陆次数'] = group_merge_all['主席登陆次数'].fillna(0)
group_merge_all['操作员登陆次数'] = group_merge_all['操作员登陆次数'].fillna(0)
#group_merge_all

In [ ]:
# check correlation
group_merge_all.corr()

In [ ]:
to_delete1 = ['代理商报备','项目，已经额并','￥','渠道成交','成交客户','已成单客户',
              '已成单','已成单客户','成单客户','成单','已成交客户',
              '成交客户','已成交','成交','直客询价','|','项目合并','渠道报备','山东会销','渠道商报备','?','？','【】','()','（）','(成)','（客户）','中间商']
to_delete2 = ['（','）','(',')']

for item in to_delete2:
    group_merge_all.CompanyName = group_merge_all.CompanyName.str.replace(item,'')

In [ ]:
group_merge_all.to_csv('F:/hst_sync_2017/source_data/CRM_data/CRM_customer_report/group_merge_all.csv')

In [ ]:
# correlation pair plot
import matplotlib as mpl
mpl.rcParams['font.serif'] = ['SimHei']
import seaborn as sns
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})
%matplotlib inline

#sns.pairplot(group_merge_all)

In [ ]:
# import crm customer record data
crm = pd.read_csv('F:/hst_sync_2017/source_data/CRM_data/CRM_customer_report/customer_report.csv')
crm.head()

In [ ]:
# change datetime column to timestamp
# groupby week,count customer report times
crm["售后时间"] = pd.to_datetime(crm["售后时间"])
group_crm= crm.set_index('售后时间').groupby(["关联客户"]).resample("W").size().to_frame(name = 'count').reset_index()
group_crm.head()

In [ ]:
to_delete1 = ['代理商报备','项目，已经额并','￥','渠道成交','成交客户','已成单客户',
              '已成单','已成单客户','成单客户','成单','已成交客户',
              '成交客户','已成交','成交','直客询价','|','项目合并','渠道报备','山东会销','渠道商报备','?','？','【】','()','（）','(成)','（客户）','中间商']
to_delete2 = ['（','）','(',')']
for item in to_delete1:
    group_crm['关联客户'] = group_crm['关联客户'].str.replace(item,'')
for item in to_delete2:
    group_crm['关联客户'] = group_crm['关联客户'].str.replace(item,'')
group_crm.关联客户.value_counts()

In [ ]:
group_crm.to_csv('F:/hst_sync_2017/source_data/CRM_data/CRM_customer_report/group_crm.csv')